<a href="https://colab.research.google.com/github/rahiakela/coursera-advanced-machine-learning-specialization/blob/course-1-introduction-to-deep-learning/week_2_tensorflow_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What is TensorFlow

In [0]:
%config IPCompleter.greedy=True

In [2]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

1.15.0


In [4]:
tf.reset_default_graph()
a = tf.placeholder(np.float32, (2, 2))
b = tf.Variable(tf.ones((2, 2)))
c = a @ b
print(c)

Tensor("matmul:0", shape=(2, 2), dtype=float32)


In [6]:
s = tf.InteractiveSession()
s.run(tf.global_variables_initializer())
s.run(c, feed_dict={a: np.ones((2, 2))})

array([[2., 2.],
       [2., 2.]], dtype=float32)

In [0]:
s.close()

# Our first model in TensorFlow

### Simple optimization (with simple prints)

In [0]:
tf.reset_default_graph()
x = tf.get_variable('x', shape=(), dtype=tf.float32, trainable=True)
f = x ** 2

In [9]:
optimizer = tf.train.GradientDescentOptimizer(0.1)
step = optimizer.minimize(f, var_list=[x])
tf.trainable_variables()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


[<tf.Variable 'x:0' shape=() dtype=float32_ref>]

In [10]:
with tf.Session() as sess:   # in this way session will be closed automatically
  sess.run(tf.global_variables_initializer())
  for i in range(10):
    _, curr_x, curr_f = sess.run([step, x, f])
    print(curr_x, curr_f)

-0.44017068 0.3027347
-0.35213655 0.19375022
-0.28170925 0.124000154
-0.2253674 0.079360105
-0.18029392 0.050790463
-0.14423513 0.032505896
-0.11538811 0.020803774
-0.09231049 0.013314416
-0.07384839 0.008521226
-0.059078712 0.0054535847


### Simple optimization (with tf.Print)

In [11]:
tf.reset_default_graph()
x = tf.get_variable('x', shape=(), dtype=tf.float32)
f = x ** 2
f = tf.Print(f, [x, f], 'x, f:')

Instructions for updating:
Use tf.print instead of tf.Print. Note that tf.print returns a no-output operator that directly prints the output. Outside of defuns or eager mode, this operator will not be executed unless it is directly specified in session.run or used as a control dependency for other operators. This is only a concern in graph mode. Below is an example of how to ensure tf.print executes in graph mode:



In [0]:
optimizer = tf.train.GradientDescentOptimizer(0.1)
step = optimizer.minimize(f)

In [0]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(10):
    sess.run([step, f])

### Simple optimization (with TensorBoard logging)

In [0]:
tf.reset_default_graph()
x = tf.get_variable("x", shape=(), dtype=tf.float32)
f = x ** 2

In [0]:
optimizer = tf.train.GradientDescentOptimizer(0.1)
step = optimizer.minimize(f)

In [0]:
tf.summary.scalar('curr_x', x)
tf.summary.scalar('curr_f', f)
summaries = tf.summary.merge_all()

In [0]:
s = tf.InteractiveSession()
summary_writer = tf.summary.FileWriter('logs/1', s.graph)
s.run(tf.global_variables_initializer())
for i in range(10):
  _, curr_summaries = s.run([step, summaries])
  summary_writer.add_summary(curr_summaries, i)
  summary_writer.flush()

**If you're running on Google Colab you can still run TensorBoard!**

In [18]:
! wget https://raw.githubusercontent.com/hse-aml/intro-to-dl/master/setup_google_colab.py -O setup_google_colab.py

--2019-12-22 10:50:33--  https://raw.githubusercontent.com/hse-aml/intro-to-dl/master/setup_google_colab.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3636 (3.6K) [text/plain]
Saving to: ‘setup_google_colab.py’

setup_google_colab. 100%[===================>]   3.55K  --.-KB/s    in 0s      

2019-12-22 10:50:33 (78.4 MB/s) - ‘setup_google_colab.py’ saved [3636/3636]



In [19]:
import setup_google_colab

# run tensorboard in background
import os
os.system('tensorboard --logdir=./logs --host 0.0.0.0 --port 6006 &')

# expose port and show the link
setup_google_colab.expose_port_on_colab(6006)

Can't get public url, retrying...
Open https://03be6fd6.ngrok.io to access your 6006 port


In [0]:
s.close()

### Training a linear model

In [23]:
# generate model data
N = 1000
D = 3
x = np.random.random((N, D))
w = np.random.random((D, 1))
y = x @ w + np.random.randn(N, 1) * 0.20

print(x.shape, y.shape)
print(w.T)

(1000, 3) (1000, 1)
[[0.7244166  0.60522309 0.5059811 ]]


In [24]:
tf.reset_default_graph()

features = tf.placeholder(tf.float32, shape=(None, D))
target = tf.placeholder(tf.float32, shape=(None, 1))

# make prediction
weights = tf.get_variable('weights', shape=(D, 1), dtype=tf.float32)
predictions = features @ weights

# compute loss
loss = tf.reduce_mean((target - predictions) ** 2)
print(target.shape, predictions.shape, loss.shape)

(?, 1) (?, 1) ()


In [0]:
optimizer = tf.train.GradientDescentOptimizer(0.1)
step = optimizer.minimize(loss)

In [26]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(300):
    _, curr_loss, curr_weights = sess.run([step, loss, weights], feed_dict={features: x, target: y})
    if i % 50 == 0:
      print(curr_loss)

2.2483761
0.06170673
0.04483554
0.04183193
0.04129515
0.04119886


In [27]:
# found weights
curr_weights.T

array([[0.72473514, 0.6086059 , 0.4939854 ]], dtype=float32)

In [28]:
# true weights
w.T

array([[0.7244166 , 0.60522309, 0.5059811 ]])

In [43]:
print(loss.shape, predictions.shape, features.shape, weights.shape)

() (?, 1) (?, 3) (3, 1)


In [0]:
tf.summary.scalar('features', features)
tf.summary.scalar('weights', weights)
summaries = tf.summary.merge_all()

In [0]:
s = tf.InteractiveSession()
summary_writer = tf.summary.FileWriter("logs/2", s.graph)
s.run(tf.global_variables_initializer())
for i in range(10):
    _, curr_summaries = s.run([step, summaries])
    summary_writer.add_summary(curr_summaries, i)
    summary_writer.flush()